# LITA Checkpoint Conversion, Finetuning and Inference Tutorial

### Note:
Currently, this notebook can be run in a NeMo container (>= 24.07). An example command to launch the container:

```
docker run --gpus all -it --rm  -v $PWD:/ws --shm-size=8g -p 8888:8888 --ulimit memlock=-1 --ulimit stack=67108864 <your_nemo_container>
```
For inference and finetuning, you need to increase the share memory size to avoid some OOM issue. For example,
```
docker run --gpus all -it --rm  -v $PWD:/ws --shm-size=128g -p 8888:8888 --ulimit memlock=-1 --ulimit stack=67108864 nvcr.io/nvidia/nemo:dev
```

By `-v $PWD:/ws`, we can mount the current local directory to `/ws/` in docker container. We may use this local directory to put the `NeMo` source code, checkpoints and dataset we will generate.

If you wanna use NeMo container (>24.04 and < 24.07) (not recommended), you need to manually mount the latest nemo:
```
docker run --gpus all -it --rm -v <your_nemo_dir>:/opt/NeMo -v $PWD:/ws --shm-size=128g -p 8888:8888 --ulimit memlock=-1 --ulimit stack=67108864 <your_nemo_container>
```

# LITA Introduction

[LITA](https://arxiv.org/pdf/2403.19046) stands for Language Instructed Temporal-Localization Assistan, which demonstrates strong performance on Reasoning Temporal Localization (RTL) task. It introduces time tokens to better help LLM understand 'When?' question in video. The below figure from [LITA paper](https://arxiv.org/pdf/2403.19046) shows a clear idea of how LITA works.

<img src="images/LITA_arch.png" alt="drawing" style="width:800px;"/>

## Tokenizer and Checkpoint Conversion
As we learned that LITA introduces `time tokens` so that timestampes of events in a video would be represented as time tokens instead of the original float point timestamps. Therefore we need to add these time tokens to the tokenizer of the backbone/LLM model. In this example, we take `Llama-3-VILA1.5-8B` as an example to show how to integrate LITA to a LLaVA like model. You may also use similar steps to convert other llama or LLaVA like models that have backbone LLM as llama such as [vicuna](https://huggingface.co/lmsys/vicuna-13b-v1.5) and [llava-v1.6-vicuna-13b](https://huggingface.co/liuhaotian/llava-v1.6-vicuna-13b).

Please download the huggingface `Llama-3-VILA1.5-8B` model.

In [ ]:
! mkdir pretrained_models && cd pretrained_models
! git clone https://huggingface.co/Efficient-Large-Model/Llama-3-VILA1.5-8B

### Tokenizer conversion
Here we show how to add 100 time tokens and some nemo extra tokens to a huggingface tokenizer.
For the definition of nemo extra tokens, please refer to `/opt/NeMo/nemo/collections/multimodal/data/neva/conversation.py`.


In [ ]:
# define the TIME_TOKEN_TEMPLATE
TIME_TOKEN_TEMPLATE = "<t{t}>"
hf_llm_model_path='/ws/pretrained_models/Llama-3-VILA1.5-8B/llm'
tokenizer_path = '/ws/converted_models/tokenizer/'

In [ ]:

import transformers
tokenizer = transformers.AutoTokenizer.from_pretrained(hf_llm_model_path)
DEFAULT_IM_START_TOKEN = "<extra_id_4>" # mark the start of the slow token
DEFAULT_IM_END_TOKEN = "<extra_id_5>" # the end of the slow token
VID_START_TOKEN = "<extra_id_8>" # the start of the fast token
VID_END_TOKEN = "<extra_id_9>" # the end of the fast token
num_new_tokens = tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, VID_START_TOKEN, VID_END_TOKEN], special_tokens=True)
tokenizer.pad_token = tokenizer.eos_token  # use eos token as pad token
num_time_tokens = 100
time_tokens = [TIME_TOKEN_TEMPLATE.format(t=x) for x in range(num_time_tokens)]
num_new_tokens = tokenizer.add_tokens(time_tokens)
# add the other nemo extra tokens
extra_tokens = ["<extra_id_0>","<extra_id_1>","<extra_id_2>","<extra_id_3>","<extra_id_6>","<extra_id_7>"]
tokenizer.add_tokens(extra_tokens)
tokenizer.save_pretrained(tokenizer_path)
print(tokenizer.vocab_size)

You can check the tokenizer by:

In [ ]:
from nemo.collections.nlp.modules.common.tokenizer_utils import get_nmt_tokenizer
tokenizer = get_nmt_tokenizer(library="huggingface", model_name=tokenizer_path)
print(tokenizer.vocab_size)

Notice if you wanna convert checkpoints trained from [LITA1.0](https://github.com/NVlabs/LITA), you should put all the extra tokens including `DEFAULT_IM_START_TOKEN` and `DEFAULT_IM_END_TOKEN` at the end of the time tokens.

### Checkpoint Conversion
Since VILA and LITA shared a similar model structure as LLaVA, we'll leverage `/opt/NeMo/examples/multimodal/multimodal_llm/neva/convert_llava_to_neva.py` for converting the checkpoint. Since VILA and LITA depends on LLaVA, we need to clone LLaVA first.


In [ ]:
! cd /opt/
! git clone --depth 1 --branch v1.2.2 https://github.com/haotian-liu/LLaVA/
! export PYTHONPATH=/opt/LLaVA/:$PYTHONPATH
! cd /ws  # do not run the below commands under `/opt` folder

In [ ]:
# check the config file in /opt/NeMo/examples/multimodal/multimodal_llm/neva/conf/vita_config.yaml
! python /opt/NeMo/examples/multimodal/multimodal_llm/neva/convert_llava_to_neva.py \
    --in-file /ws/pretrained_models/Llama-3-VILA1.5-8B/llm \
    --mm-vision-tower /ws/pretrained_models/Llama-3-VILA1.5-8B/vision_tower \
    --mm-projector-ckpt-dir /ws/pretrained_models/Llama-3-VILA1.5-8B/mm_projector \
    --out-file /ws/converted_models/Llama-3-VILA1.5-8B.nemo \
    --tokenizer-model /ws/converted_models/tokenizer/ \
    --config-file vita_config.yaml \
    --model-type VITA \
    --conv-template llama_3

Notice if `mm_vision_tower` can be downloaded from huggingface and you don't want to change it, then you don't need to explicitly add this option.  And similarly, only when you want to change the `mm_projector`, you will need to add the `mm_projector_ckpt_dir`.


## Finetuning

In this section, we'll preprocess the Dense Videco Captioning dataset and then do finetuning with the nemo ckpt we just converted.

### Convert Dataset
The targeted dataset file format for finetuning should be like:
```bash
[
    # 1st example: video question answer
    {
        "id": "1043215450",
        "video": "076101_076150/1043215450.mp4",   # video_path will be prepended
        "conversations": 
        [
            {"from": "human", "value": "<video>\n is the athlete wearing trousers"}, 
            {"from": "gpt", "value": "Yes"}
        ]       
    },
    # 2nd example: dense video captioning
    {
        "id": "xxxx",
        "video: "xxxx.mp4",
        "conversations":
        [
            {"from": "human", "value": "<video>\n "Provide a detailed description of the given video.Prepend each sentence with its start and end timestamps."}, 
            {"from": "gpt", "value": "<t1> <t2> Apply eyeshadow on the crease with brush <t3> <t4> Apply eyeshadow on the outer corner of eyes with brush"}
        ]
    },
    # 3rd example: event classification
    {
        "id": "xxxx",
        "video: "xxxx.mp4",
        "conversations":
        [
            {"from": "human", "value": "<video>\n "What is the action performed in this video?"}, 
            {"from": "gpt", "value": "brush hair"}
        ]

    },
    # 4th example: event localization
    {
        "id": "-4RXOT_UfpM_2",
        "video": "-4RXOT_UfpM_2.mp4",
        "conversations": [
            {"from": "human", "value": "<video>\nWhen is \"Apply concealer on the eyelids and blend with sponge\" depicted in the video? Provide a response using only start and end timestamps."},
            {"from": "gpt", "value": "<t4> <t18>"}
        ],
        "durations": 119.01901901901903
    },
    ...
]
```

Here the `<video>` is the placeholder for the video features. In the 2nd example, `<t1>` `<t2>` are the time tokens to indidate in which time interval we've seen this event or description of the time inverval. You can prepare your time tokens like this:


In [ ]:
import numpy as np
TIME_TOKEN_TEMPLATE = "<t{t}>"
def time_to_string(time, num_time_tokens):
    max_offset = float(num_time_tokens - 1)
    time = int(np.round(max_offset * time))
    return TIME_TOKEN_TEMPLATE.format(t=time)

# example of converting time tokens
# from 10seconds to 15 seconds
num_time_tokens = 100
start = 10.0   # the 10 seconds
end = 15.0     # the 15 seconds
duration = 200.0 # total video duration is 200seconds
start = start / duration 
end = end / duration
start_time_token_str = time_to_string(start, num_time_tokens)
end_time_token_str = time_to_string(end, num_time_tokens,)

For Dense Video Captioning (DVC) task or Resoning Temporal Localization (RTL) task, your dataset probably looks like:
```bash
{
    "video_name": {
        "duration": 125.0,
        "timestamps": [
            [0, 5], 
            [3, 9]
        ],
        "sentences": [
            "Here is your caption 1",
            "Here is your caption 2",
        ],
        "events": [
            "Event 1",
            "Event 2",
        ]
    },
    ...
}
```

If you've already prepared this style dataset, you may refer to `convert_dvc_dataset_for_training.py`, `convert_dvc_dataset_for_evaluation.py` and `convert_video_qa_dataset.py` under `NeMo/scripts/multimodal_dataset_conversion` to convert the datasets so that they could be used in finetuning. If you want to augment your dataset by leveraging the NVIDIA LLM APIs or external LLMs, you may refer to `generate_qa_data.py` under the same directory.

We take the `YouMakeUp` dataset as an example.
1. First download dataset and prepare it to DVC dataset format.

In [ ]:
%%bash
git clone https://github.com/AIM3-RUC/YouMakeup/tree/master
pip install yt-dlp==2024.4.9
pip install moviepy

#download videos, this may take a while
python /opt/NeMo/scripts/multimodal_dataset_conversion/prepare_youmakeup.py -i YouMakeup/data/train/train_steps.json -o /ws/dataset -d True

#chunk videos into clips, with each clip containing 120 seconds
python /opt/NeMo/scripts/multimodal_dataset_conversion/prepare_youmakeup.py -i YouMakeup/data/train/train_steps.json -o /ws/dataset -l 12

#create evaluation dataset
python /opt/NeMo/scripts/multimodal_dataset_conversion/prepare_youmakeup.py -i YouMakeup/data/valid/valid_steps.json -o /ws/dataset/valid/ -d True
python /opt/NeMo/scripts/multimodal_dataset_conversion/prepare_youmakeup.py -i YouMakeup/data/train/valid_steps.json -o /ws/dataset/valid/ -l 120

#create QA style validation/evaluation or test dataset
python3 /opt/NeMo/scripts/multimodal_dataset_conversion/convert_dvc_dataset_for_evaluation.py --input /ws/dataset/valid/train.json --output_file=/ws/dataset/valid/rtl_eval.json

2. Your dataset should now have the following structure:
```bash
    dataset/
        videos/
            video1.mp4
            video2.mp4
            ...
        train.json
```
Then we convert the dataset to finetuning format by:

In [ ]:
%%bash
# generate custom caption dataset and multiply the dataset by three times
python /opt/NeMo/scripts/multimodal_dataset_conversion/convert_dvc_dataset_for_training.py \
    --input_dvc_dataset /ws/dataset/train.json \
    --video_path_prefix /ws/dataset/videos/ \
    --subtask custom_caption --data_multiplier 3 \
    --output_file /ws/dataset/vc_train.json

# generate event loalization dataset and increase the dataset by three times
python /opt/NeMo/scripts/multimodal_dataset_conversion/convert_dvc_dataset_for_training.py \
    --input_dvc_dataset /ws/dataset/train.json \
    --video_path_prefix /ws/dataset/videos/ \
    --subtask event_localization --data_multiplier 3 \
    --output_file /ws/dataset/event_loc_train.json

In [ ]:
# Let's combine the two json files into one
import json
def get_list_data(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data
l1 = get_list_data('/ws/dataset/vc_train.json')
l2 = get_list_data('/ws/dataset/event_loc_train.json')
l = l1 + l2
# shuffle the data
import random
random.shuffle(l)
with open('/ws/dataset/combined_train.json', 'w') as f:
    json.dump(l, f, indent=4)

### Finetuning
We'll use 8xA100(80GB) to do the finetuning. You may set smaller number of `num_frames` or change the `tensor_model_parallel_size` or `pipeline_model_parallel_size` if you encounter OOM issue.

Run finetuning by:

In [ ]:
%%bash
video_folder=/ws/dataset/videos/
data_path=/ws/dataset/combined_train.json
model_path=/ws/converted_models/Llama-3-VILA1.5-8B.nemo
EXP_MANAGER_DIR=/ws/train  # check this directory for experiment details
num_gpus=8
torchrun --nproc_per_node=${num_gpus} /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_finetune.py \
  --config-path=/opt/NeMo/examples/multimodal/multimodal_llm/neva/conf/ \
  --config-name=vita_config.yaml \
  ++cluster_type=BCP \
  trainer.num_nodes=1 \
  trainer.precision=bf16 \
  trainer.devices=${num_gpus} \
  trainer.max_steps=262 \
  trainer.limit_val_batches=5 \
  model.megatron_amp_O2=false \
  model.mm_cfg.llm.freeze=false \
  model.mm_cfg.vision_encoder.freeze=true \
  model.mm_cfg.vision_encoder.from_pretrained=/ws/pretrained_models/Llama-3-VILA1.5-8B/vision_tower \
  model.mm_cfg.vision_encoder.model_type=siglip_vision_model \
  model.global_batch_size=128 \
  model.micro_batch_size=1 \
  model.tensor_model_parallel_size=4 \
  model.pipeline_model_parallel_size=1 \
  model.restore_from_path=${model_path} \
  model.context_parallel_size=1 \
  model.data.video_folder=${video_folder} \
  model.data.data_path=${data_path} \
  model.data.num_frames=128 \
  model.mm_cfg.use_lita=true \
  model.mm_cfg.lita.lita_video_arch=temporal_all_resolution \
  model.mm_cfg.lita.visual_token_format=im_vid_start_end \
  model.mm_cfg.lita.sample_frames=4 \
  model.mcore_gpt=true \
  model.transformer_engine=true \
  model.optim.sched.warmup_steps=8 \
  exp_manager.create_checkpoint_callback=True \
  exp_manager.create_wandb_logger=False \
  exp_manager.wandb_logger_kwargs.project=neva_lita \
  exp_manager.wandb_logger_kwargs.name=neva_lita_finetuning \
  exp_manager.exp_dir=${EXP_MANAGER_DIR}

## Evaluation
After training for around 8 hours, we can split the `rtl_eval.json` file into `$num_gpus` number of input files so that the inference can be accelerated.
We can do this by:

In [ ]:
import os
json_file="/ws/dataset/valid/rtl_eval.json"
num_splits=8
output_dir="/ws/dataset/valid/split/"
os.makedirs(output_dir, exist_ok=True)
with open(json_file, 'r') as f:
    data = json.load(f)
data = sorted(data, key=lambda x: x['video']) # group by video
num_samples = len(data)
split_size = num_samples // num_splits
for i in range(num_splits):
    start = i * split_size
    end = (i + 1) * split_size if i < num_splits - 1 else num_samples
    split_data = data[start:end]
    output_file = os.path.join(output_dir, f"{i}.jsonl")
    with open(output_file, 'w') as f:
        json.dump(split_data, f)
    print(f"Saved {end - start} samples to {output_file}")

Now we can run the bash script：

In [ ]:
%%bash
neva_model_file=/ws/train/neva_lita_finetuning.nemo

num_gpus=8
num_splits=$num_gpus

mkdir -p /ws/dataset/valid/split_output

for i in $(seq 0 $((num_splits-1)))
do
    num_gpus=1
    # For single input file and single gpu,
    # please remove the for loop and set the prompt_file and output_file directly
    # prompt_file=/ws/dataset/valid/rtl_eval.json
    # output_file=/ws/dataset/valid/nemo_infer_output.json
    prompt_file=/ws/dataset/valid/split/$i.json
    output_file=/ws/dataset/valid/split_output/nemo_infer_output_$i.json
    video_base_path=/ws/dataset/valid/videos/
    sleep 1
    CUDA_VISIBLE_DEVICES=$i torchrun --nnodes=1 --standalone /ws/NeMo/examples/multimodal/multimodal_llm/neva/neva_evaluation.py \
               --config-path=/opt/NeMo/examples/multimodal/multimodal_llm/neva/conf/ \
               --config-name=neva_inference.yaml \
               tensor_model_parallel_size=1 \
               pipeline_model_parallel_size=1 \
               neva_model_file=$neva_model_file \
               trainer.devices=$num_gpus \
               trainer.precision=bf16 \
               prompt_file=$prompt_file \
               inference.media_base_path=$video_base_path \
               inference.media_type=video \
               output_file=$output_file \
               inference.temperature=0.2 \
               inference.top_k=0 \
               inference.top_p=0.9 \
               inference.greedy=True \
               +add_media_sep=true \
               inference.end_strings='["<|eot_id|>"]' \
               inference.add_BOS=False \
               inference.all_probs=False \
               inference.repetition_penalty=1.2 \
               inference.insert_media_token=left \
               inference.tokens_to_generate=256 \
               +inference.batch_size=8 \
               quantization.algorithm=awq \
               quantization.enable=False \
    &
done
wait


In [ ]:
# combine the output json files if you splitted them before
input_dir = "/ws/dataset/valid/split_output/"
output_file = "/ws/dataset/valid/split_output/nemo_infer_output_total.json"
data = []
for file in os.listdir(input_dir):
    if file.endswith(".json"):
        with open(os.path.join(input_dir, file), "r") as f:
            data.extend(json.load(f))
print(f"Total number of items: {len(data)}")
with open(output_file, "w") as f:
    json.dump(data, f, indent=4)

Finally, we can do the evaluation on RTL task by:

In [ ]:
%%bash
python3 /opt/NeMo/examples/multimodal/multimodal_llm/neva/eval/eval_video_rtl.py \
    --input_file=/ws/dataset/valid/split_output/nemo_infer_output_total.json \
    --output_dir=/ws/dataset/valid/split_output/ --save_mid_result

You many also refer to `/opt/NeMo/examples/multimodal/multimodal_llm/neva/eval/eval_vqa.py` to check how to use external LLM API to do the video question answering task evaluation.